# YOLOv5 Model Assisted Labeling

> Work based on https://models.roboflow.com/object-detection/yolov5.

Labeling a new dataset from scratch is tiresome. To label images faster, I train a YOLOv5 model and apply its predictions to new images.

## How to use

### Vanilla image sets

In [ ]:
%%capture
#hide
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7tGbDc1dmqMisGRCMTBZ-afkJknMexgtpAYx9iFpZaoD3J3G2AScw


In [ ]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"
%run '_Synch.ipynb'

/content/drive/MyDrive/Coding/ModelAssistedLabel


Recursively search a folder (`repo`) that contains images and labels.

In [ ]:
from ModelAssistedLabel.core import Generation, Defaults
g = Generation(repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)", 
              out_dir = ".",
              data_yaml=Defaults().data_yaml)

def tostr(split):
  return [{k: len(v)} for k,v in split.items()]

def sumg(split):
  return sum([list(x.values())[0] for x in tostr(split)])

for MAX_SIZE in [10, None]:
  g.set_split(MAX_SIZE=MAX_SIZE)
  print("summary: ", tostr(g.split))
  print("checksum:", sumg(g.split))

summary:  [{'train': 7}, {'valid': 2}, {'test': 1}]
checksum: 10
summary:  [{'train': 589}, {'valid': 169}, {'test': 83}]
checksum: 841


Writing these files to disk for when we need them later.

In [ ]:
zipped = g.process_split(" vanilla")

In [ ]:
import os

local = os.path.basename(zipped)
!unzip "{local}" #grab data
!rm -f -r '{local}' #remove zip file
mv '{local[:-4]}/data.yaml' .

In [ ]:
ls

 00_ultralytics.ipynb   Makefile
 01_split.ipynb         MANIFEST.in
 02_augment.ipynb       ModelAssistedLabel/
 CONTRIBUTING.md        README.md
 data.yaml             'Roboflow Export (841) vanilla 21-03-15 20-18-11'/
 docker-compose.yml     settings.ini
 docs/                  setup.py
 index.ipynb            _Synch.ipynb
 LICENSE                yolov5/


In [ ]:
from ModelAssistedLabel.core import Trainer

ImportError: ignored